In [76]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import brightway2 as bw

# Custom utils defined for inter-acv
from utils import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [85]:
# Set project 
bw.projects.set_current('B_Publication')
initDb()

In [86]:
# Select impacts
impacts = [m for m in bw.methods if 'ILCD 1.0.8 2016' in str(m) and 'no LT' in str(m) ]

In [87]:
us_elec = getActivity('market group for electricity, medium voltage', 'US')
glo_elec = getActivity('market group for electricity, medium voltage', 'GLO')

In [98]:
foo = newActivity("foo", "kg", exchanges={us_elec:1, glo_elec:"alpha"})
print_act(foo)

Activity 'foo' was already in 'incer-acv'. Overwriting it


,amount,unit,type
"'market group for electricity, medium voltage' (kilowatt hour, US, None)",1,kilowatt hour,technosphere
"'market group for electricity, medium voltage' (kilowatt hour, GLO, None)",0,kilowatt hour,technosphere


In [114]:
bar = newActivity("bar", "kg")

Activity 'bar' was already in 'incer-acv'. Overwriting it


In [130]:
elect_switch_param = newParamDef('elect_switch_param', ParamType.ENUM, "us", values=["us", "glo"], description="Switch on electricty mix")
alpha = newParamDef('alpha', ParamType.NUMBER, 0.5, description="alpha")

In [131]:
elec_switch_activity = newSwitchActivity(elect_switch_param, dict(us=us_elec, glo=glo_elec))
print_act(elec_switch_activity)

Activity 'elect_switch_param switch' was already in 'incer-acv'. Overwriting it


,amount /formula,unit,type
"'market group for electricity, medium voltage' (kilowatt hour, US, None)",elect_switch_param_us,kilowatt hour,technosphere
"'market group for electricity, medium voltage' (kilowatt hour, GLO, None)",elect_switch_param_glo,kilowatt hour,technosphere


In [133]:
computeLCA(elec_switch_activity, impacts, amount=0.1, elect_switch_param="glo")


KeyError: 'elect_switch_param'